### References:

Darmet, L., Ladouce, S., & Dehais, F. (2023). **Shortened calibration of code-VEP based BCI by improved deep learning 
architecture and golden subjects pre-training.** *In 11th International IEEE EMBS Conference on Neural Engineering.* 

Source: https://github.com/neuroergoISAE/offline_cVEP_bitwise

Thielen, J., Marsman, P., Farquhar, J., & Desain, P. (2021). **From full calibration to zero training for a code-modulated
visual evoked potentials for brain–computer interface**. *Journal of Neural Engineering, 18(5), 056007.*

Source: https://gitlab.socsci.ru.nl/jthielen/pynt/-/blob/main/examples

Sebastian Nagel

Source: https://github.com/thecodingdad/eeg2code

Teulings, D.E.J. David (2022).  **an-application-of-recurrency-to-eeg2code-deep-learning**

Source: https://gitlab.socsci.ru.nl/D.Teulings/an-application-of-recurrency-to-eeg2code-deep-learning/-/tree/master

### Hard code that need check for dataset location

path = 'D:/Dataset'

In [1]:
# the path to the dataset
path = 'D:/Dataset'

In [2]:
import os
import numpy as np
import json
import pandas as pd
import matplotlib.pyplot as plt
import pynt
import keras
import sys
import scipy.io as sio
import tensorflow as tf
import time
import seaborn
import random
import keras.utils as ku
from scipy import stats
from scipy.stats import wilcoxon
from keras.optimizers import adam
from keras.callbacks import History 
from sklearn.metrics import accuracy_score
from tensorflow.python.keras import backend
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from keras import optimizers
from keras import initializers
from keras.models import load_model
from keras.models import Sequential
from keras.layers import (Input,
                          InputLayer,
                          GRU,
                          RNN,
                          LSTM,
                          Reshape,
                          SimpleRNN,
                          Conv1D,
                          Conv2D,
                          MaxPooling1D,
                          MaxPooling2D,
                          Permute,
                          Flatten, 
                          Dense,
                          BatchNormalization, 
                          Activation,
                          Dropout
                         )
from scipy.io import loadmat
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.cross_decomposition import CCA
from tensorflow.python.keras import backend
from tensorflow.keras.models import load_model

# Settings
### Hyperparameters, function that load data, and function that slicing data to epoch

In [3]:
"""
Hyperparameters.

subjects = ['sub-01','sub-02', 'sub-03', 'sub-04', 'sub-05','sub-06','sub-07', 'sub-08', 'sub-09', 'sub-10'
           ,'sub-11','sub-22', 'sub-13', 'sub-14', 'sub-15','sub-16','sub-17', 'sub-18', 'sub-19', 'sub-20'
           ,'sub-21','sub-22', 'sub-23', 'sub-24', 'sub-25','sub-26','sub-27', 'sub-28', 'sub-29', 'sub-30']
epoch_size: 0.3 * 300 sampling frequency = 90 window size
step_size: 1 / 60 frame rate * 300 sampling frequency = 5
learning rate: 0.001 same as EEG2Code paper
batch size: 256 same as EEG2Code paper
    Mini-batch gradient descent
epochs: 30 (25 on EE2Code paper)
    Number of iterations
Optimizers: RMSProp
n_folds: 5 k-fold cross validation
"""
subjects =  ['sub-01','sub-02', 'sub-03', 'sub-04', 'sub-05','sub-06','sub-07', 'sub-08', 'sub-09', 'sub-10'
           ,'sub-11','sub-22', 'sub-13', 'sub-14', 'sub-15','sub-16','sub-17', 'sub-18', 'sub-19', 'sub-20'
           ,'sub-21','sub-22', 'sub-23', 'sub-24', 'sub-25','sub-26','sub-27', 'sub-28', 'sub-29', 'sub-30']
epoch_size = int(300 * 0.3)
step_size = int(1 / 60 * 300)
learning_rate = 0.001
batch_size = 256
epochs = 30
adam = keras.optimizers.RMSprop(learning_rate=learning_rate)
n_folds = 5

In [4]:
"""
Pre-processed settings

band_pass: [2.0 Hz, 30.0 Hz]
    Bandpass cutoffs in Hz for spectral filtering
tmin: 0.0 sec
    Trial onset in seconds for slicing
tmax: 31.5 sec
    Trial end in seconds for slicing
    
Load data.

X: (100 trials, 8 channels, 9450 samples)
    EEG time-series data
V: (630 samples, 20 classes/labels/symbols)
    Codes matrix. Codebook
y: (100 trials,)
    Label vector of 100 trials
fs = 300 Hz
    Target sampling frequency (avg of sampels obtained in one sec) in Hz for resampling
"""
def load_data(subject):
    fn = os.path.join(path, 'derivatives300', 'offline', subject, f'{subject}_gdf.npz')
    tmp = np.load(fn)
    X = tmp['X']
    V = tmp['V']
    y = tmp['y']
    fs = tmp['fs']
    return X, V, y, fs

In [5]:
"""
Slices trials to epochs.

Returns:
    X_sliced: (100 trials, 1872 epochs, 8 channels, 90 window size)
        The sliced EEG data of shape with epoch
    y_sliced: (100 trials, 1872 epochs)
        The sliced label information of shape with epoch
"""
def trials_to_epochs(X, y, codes, epoch_size, step_size):
    n_trials, n_channels, n_samples = X.shape
    n_epochs = int((n_samples - epoch_size) / step_size)
    X_sliced = np.zeros((n_trials, n_epochs, n_channels, epoch_size), 
        dtype='float32')
    y_sliced = np.zeros((n_trials, n_epochs), dtype='uint8')
    for i_epoch in range(n_epochs):
        start = i_epoch * step_size
        X_sliced[:, i_epoch, :, :] = X[:, :, start:start+epoch_size]
        y_sliced[:, i_epoch] = codes[start % codes.shape[0], y]
    return X_sliced, y_sliced

# Method
### EEG2Code

In [6]:
"""
EEG2Code Architecture

Difference from paper
    Maxpooling 1D(1 pool size , 1 strides) -> Maxpooling2D((1, 1), (1, 1))
    Conv1d(kernel size) -> Conv2d((1,kernel size))
    Dense(1024) -> Dense(256)
    Softmax -> Sigmoid
"""
def EEG2Code(n_channels, epoch_size):
    model = Sequential()
    # layer 1
    model.add(Conv2D(16, input_shape=(n_channels, epoch_size, 1),
        kernel_size=(n_channels, 1),
        padding='valid',
        strides=(1, 1)))
    model.add(BatchNormalization(axis=3, scale=False, center=False))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(
        pool_size=(1, 1), # 2,1
        padding='same', 
        strides=(1, 1))) 
    # layer 2
    model.add(Conv2D(8, 
        kernel_size=(1, 64),
        padding='same', 
        strides=(1, 1)))
    model.add(BatchNormalization(axis=3, scale=False, center=False))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(
        pool_size=(1, 1), # 2,1
        padding='same', 
        strides=(1, 1))) 
    model.add(Dropout(0.5))
    # layer 3
    model.add(Conv2D(4,
        kernel_size=(1, 5),
        padding='same',
        strides=(1, 1)))
    model.add(BatchNormalization(axis=3, scale=False, center=False))               
    model.add(Activation('relu'))
    model.add(MaxPooling2D(
        pool_size=(1, 1), # 2,1
        padding='same', 
        strides=(1, 1)))
    model.add(Dropout(0.5))
    # layer 4
    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    # layer 5
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model

In [7]:
"""
Train, test, evaluate EEG2Code
X_sliced: (100 trials, 1872 epochs, 8 channels, 90 window size)
X_sliced_reshape: (187200 trials*epochs, 8 channels, 90 window size, 1)
y_sliced: (100 trials, 1872 epochs)
y_sliced_reshape: (187200 trials*epochs,)
_V: (20 symbols, 1872 epochs)
"""
# Load data only for n_trials, n_channels, n_samples
X1, V1, y1, fs1 = load_data(subjects[0])
n_trials, n_channels, n_samples = X1.shape

print("X: ", X1.shape)
print("V: ", V1.shape)
print("y: ", y1.shape)
print("fs: ", fs1)

# Pre-defined sequence of indices for fold assignments
folds_ntrial = np.repeat(np.arange(n_folds), n_trials / n_folds)
folds_ntrial_epochs = np.repeat(np.arange(n_folds), int(n_trials*(n_samples - epoch_size) / step_size / n_folds))

# Store accuracies for each subjects
EEG2Code_acc_epoch = []
EEG2Code_acc_trial = []

# Store epoch predictions of all fold of all subjects
epoch_predictions_saveall_eeg2code = np.empty((len(subjects), n_folds, int((n_samples - epoch_size) / step_size * (n_trials / n_folds))))

#Loop over all subjects
for j, subject in enumerate(subjects):
    # Load data
    X, V, y, fs = load_data(subject)
    print('Subject: ', subject)
    
    # Set up dataset for epoch classification
    X_sliced, y_sliced = trials_to_epochs(X, y, V, epoch_size, step_size)
    X_sliced_reshape = X_sliced.reshape(-1, X_sliced.shape[2], X_sliced.shape[3], 1)
    y_sliced_reshape = y_sliced.flatten()
    
    # Set up codebook for trial classification
    n = int(np.ceil(n_samples / V.shape[0]))    
    _V = np.tile(V, (n, 1)).astype('float32')[:n_samples-epoch_size:step_size]
    _V = _V.transpose(1,0)

    # Store accuracies for each fold
    accuracy_epoch_eeg2code = np.zeros(n_folds)
    accuracy_trial_eeg2code = np.zeros(n_folds)
    
    # Store epoch predictions of all folds
    epoch_predictions_save_eeg2code= np.empty((n_folds,int((n_samples - epoch_size) / step_size *n_trials/n_folds)))
    
    # Loop over 5-fold cross validation
    for i_fold in range(n_folds):
        # Split data to train and valid set
        X_train = X_sliced_reshape[folds_ntrial_epochs  != i_fold,:]
        X_test1 = X_sliced_reshape[folds_ntrial_epochs  == i_fold,:]
        y_train = y_sliced_reshape[folds_ntrial_epochs  != i_fold]
        y_test = y_sliced_reshape[folds_ntrial_epochs  == i_fold]
        test_y = y[folds_ntrial  == i_fold]

        # Create a new instance of EEG2Code model
        n_channels = X_train.shape[1]
        eeg2code = EEG2Code(n_channels, epoch_size)
        eeg2code.compile(
            loss=keras.losses.BinaryCrossentropy(from_logits=False),
            optimizer=adam,metrics=['accuracy'])
        #Print model structure
        if(subject == 'sub-01' and i_fold == 0):
            print("X_sliced: ", X_sliced.shape)
            print("y_sliced: ", y_sliced.shape)
            print("X_sliced_reshape: ", X_sliced_reshape.shape)
            print("y_sliced_reshape: ", y_sliced_reshape.shape)
            print("_V: ", _V.shape)
            for i in eeg2code.layers:
                print(i.name, "(', i.__class__.__name__, ')")
                print('     Input shape : ',i.input_shape)
                print('     Output shape: ', i.output_shape)
                print('     Trainable Weights(Param #): ', i.count_params())
                
        # Split data into training and validation set
        X_tr1, X_val1, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=True, random_state=None)
        
        # Normalize X training data and apply same parameter to validation and test dataset
        norm_mean = np.mean(X_tr1)
        norm_std = np.std(X_tr1)
        X_tr = (X_tr1 - norm_mean) / norm_std
        X_val = (X_val1 - norm_mean) / norm_std
        X_test = (X_test1 - norm_mean) / norm_std

        # Normalize training data and train model
        history = eeg2code.fit(X_tr, y_tr, batch_size=batch_size, epochs=epochs, 
                               validation_data = (X_val, y_val), shuffle=True,verbose=2)
        
        # Save training history
        hist_df = pd.DataFrame(history.history)
        if not os.path.exists(os.path.join(path, 'history2')):
            os.makedirs(os.path.join(path, 'history2'))
        with open(os.path.join(path, 'history2', f'{subject}_fold_{i_fold+1}_EEG2Codehistory.json'), 'w') as f:
             hist_df.to_json(f)
        
        # Evaluate accuracy on epoch level (predict epoch probability)
        epoch_predictions = eeg2code.predict(X_test)
        epoch_threshhold = (epoch_predictions > 0.5).astype(int)
        accuracy_epoch_eeg2code[i_fold] = 100*np.mean(y_test == epoch_threshhold.flatten())
        
        # Append epoch predictions for fold
        epoch_predictions_save_eeg2code[i_fold] = np.array(epoch_predictions.flatten())
        
        # Evaluate accuracy on trial level (predict sequence)
        trial_predictions = epoch_predictions.reshape(20,1872)
        # Use correlation to match predicted pattern to specific symbol
        predictions = np.zeros(20)
        for i in range(20):
            rho = np.corrcoef(trial_predictions[i,:],_V)[0, 1:]
            predictions[i]=np.argmax(rho)
        sum_value = sum(predictions == test_y)
        trial_score = sum_value/20*100
        accuracy_trial_eeg2code[i_fold] = trial_score
        
        print(f"fold: {i_fold+1} is complete")
        
    # Append the averaged fold accuracies for the subject
    EEG2Code_acc_epoch.append(accuracy_epoch_eeg2code.mean())
    EEG2Code_acc_trial.append(accuracy_trial_eeg2code.mean())
                                      
    # Append epoch predictions of all folds for subject
    epoch_predictions_saveall_eeg2code[j] = epoch_predictions_save_eeg2code

X:  (100, 8, 9450)
V:  (630, 20)
y:  (100,)
fs:  300
Subject:  sub-01
X_sliced:  (100, 1872, 8, 90)
y_sliced:  (100, 1872)
X_sliced_reshape:  (187200, 8, 90, 1)
y_sliced_reshape:  (187200,)
_V:  (20, 1872)
conv2d (', i.__class__.__name__, ')
     Input shape :  (None, 8, 90, 1)
     Output shape:  (None, 1, 90, 16)
     Trainable Weights(Param #):  144
batch_normalization (', i.__class__.__name__, ')
     Input shape :  (None, 1, 90, 16)
     Output shape:  (None, 1, 90, 16)
     Trainable Weights(Param #):  32
activation (', i.__class__.__name__, ')
     Input shape :  (None, 1, 90, 16)
     Output shape:  (None, 1, 90, 16)
     Trainable Weights(Param #):  0
max_pooling2d (', i.__class__.__name__, ')
     Input shape :  (None, 1, 90, 16)
     Output shape:  (None, 1, 90, 16)
     Trainable Weights(Param #):  0
conv2d_1 (', i.__class__.__name__, ')
     Input shape :  (None, 1, 90, 16)
     Output shape:  (None, 1, 90, 8)
     Trainable Weights(Param #):  8200
batch_normalization_1 ('

Epoch 11/30
468/468 - 26s - loss: 0.5230 - accuracy: 0.7454 - val_loss: 0.4789 - val_accuracy: 0.7753 - 26s/epoch - 57ms/step
Epoch 12/30
468/468 - 26s - loss: 0.5199 - accuracy: 0.7452 - val_loss: 0.4780 - val_accuracy: 0.7762 - 26s/epoch - 55ms/step
Epoch 13/30
468/468 - 26s - loss: 0.5185 - accuracy: 0.7481 - val_loss: 0.4752 - val_accuracy: 0.7819 - 26s/epoch - 56ms/step
Epoch 14/30
468/468 - 26s - loss: 0.5173 - accuracy: 0.7488 - val_loss: 0.4724 - val_accuracy: 0.7801 - 26s/epoch - 56ms/step
Epoch 15/30
468/468 - 27s - loss: 0.5171 - accuracy: 0.7490 - val_loss: 0.4729 - val_accuracy: 0.7805 - 27s/epoch - 57ms/step
Epoch 16/30
468/468 - 26s - loss: 0.5156 - accuracy: 0.7496 - val_loss: 0.4708 - val_accuracy: 0.7833 - 26s/epoch - 56ms/step
Epoch 17/30
468/468 - 26s - loss: 0.5134 - accuracy: 0.7504 - val_loss: 0.4735 - val_accuracy: 0.7823 - 26s/epoch - 56ms/step
Epoch 18/30
468/468 - 26s - loss: 0.5123 - accuracy: 0.7525 - val_loss: 0.4706 - val_accuracy: 0.7826 - 26s/epoch - 56

Epoch 15/30
468/468 - 26s - loss: 0.5158 - accuracy: 0.7493 - val_loss: 0.4708 - val_accuracy: 0.7823 - 26s/epoch - 55ms/step
Epoch 16/30
468/468 - 26s - loss: 0.5155 - accuracy: 0.7504 - val_loss: 0.4718 - val_accuracy: 0.7814 - 26s/epoch - 55ms/step
Epoch 17/30
468/468 - 26s - loss: 0.5152 - accuracy: 0.7503 - val_loss: 0.4685 - val_accuracy: 0.7827 - 26s/epoch - 56ms/step
Epoch 18/30
468/468 - 26s - loss: 0.5145 - accuracy: 0.7507 - val_loss: 0.4667 - val_accuracy: 0.7856 - 26s/epoch - 56ms/step
Epoch 19/30
468/468 - 25s - loss: 0.5130 - accuracy: 0.7522 - val_loss: 0.4658 - val_accuracy: 0.7837 - 25s/epoch - 54ms/step
Epoch 20/30
468/468 - 26s - loss: 0.5117 - accuracy: 0.7545 - val_loss: 0.4653 - val_accuracy: 0.7839 - 26s/epoch - 56ms/step
Epoch 21/30
468/468 - 27s - loss: 0.5118 - accuracy: 0.7536 - val_loss: 0.4675 - val_accuracy: 0.7859 - 27s/epoch - 58ms/step
Epoch 22/30
468/468 - 26s - loss: 0.5124 - accuracy: 0.7522 - val_loss: 0.4670 - val_accuracy: 0.7858 - 26s/epoch - 57

Epoch 19/30
468/468 - 43s - loss: 0.6629 - accuracy: 0.5986 - val_loss: 0.6585 - val_accuracy: 0.6073 - 43s/epoch - 92ms/step
Epoch 20/30
468/468 - 43s - loss: 0.6626 - accuracy: 0.5979 - val_loss: 0.6583 - val_accuracy: 0.6082 - 43s/epoch - 93ms/step
Epoch 21/30
468/468 - 42s - loss: 0.6619 - accuracy: 0.5988 - val_loss: 0.6580 - val_accuracy: 0.6068 - 42s/epoch - 89ms/step
Epoch 22/30
468/468 - 43s - loss: 0.6619 - accuracy: 0.5997 - val_loss: 0.6578 - val_accuracy: 0.6047 - 43s/epoch - 93ms/step
Epoch 23/30
468/468 - 44s - loss: 0.6615 - accuracy: 0.5988 - val_loss: 0.6573 - val_accuracy: 0.6060 - 44s/epoch - 93ms/step
Epoch 24/30
468/468 - 43s - loss: 0.6613 - accuracy: 0.5994 - val_loss: 0.6543 - val_accuracy: 0.6088 - 43s/epoch - 93ms/step
Epoch 25/30
468/468 - 43s - loss: 0.6609 - accuracy: 0.6014 - val_loss: 0.6564 - val_accuracy: 0.6073 - 43s/epoch - 93ms/step
Epoch 26/30
468/468 - 41s - loss: 0.6610 - accuracy: 0.6005 - val_loss: 0.6549 - val_accuracy: 0.6085 - 41s/epoch - 88

Epoch 23/30
468/468 - 25s - loss: 0.6598 - accuracy: 0.6033 - val_loss: 0.6541 - val_accuracy: 0.6131 - 25s/epoch - 54ms/step
Epoch 24/30
468/468 - 25s - loss: 0.6595 - accuracy: 0.6049 - val_loss: 0.6522 - val_accuracy: 0.6123 - 25s/epoch - 54ms/step
Epoch 25/30
468/468 - 25s - loss: 0.6588 - accuracy: 0.6040 - val_loss: 0.6553 - val_accuracy: 0.6075 - 25s/epoch - 54ms/step
Epoch 26/30
468/468 - 25s - loss: 0.6589 - accuracy: 0.6041 - val_loss: 0.6524 - val_accuracy: 0.6150 - 25s/epoch - 54ms/step
Epoch 27/30
468/468 - 25s - loss: 0.6595 - accuracy: 0.6040 - val_loss: 0.6525 - val_accuracy: 0.6143 - 25s/epoch - 54ms/step
Epoch 28/30
468/468 - 25s - loss: 0.6587 - accuracy: 0.6063 - val_loss: 0.6524 - val_accuracy: 0.6126 - 25s/epoch - 54ms/step
Epoch 29/30
468/468 - 24s - loss: 0.6578 - accuracy: 0.6070 - val_loss: 0.6547 - val_accuracy: 0.6113 - 24s/epoch - 51ms/step
Epoch 30/30
468/468 - 24s - loss: 0.6583 - accuracy: 0.6064 - val_loss: 0.6526 - val_accuracy: 0.6138 - 24s/epoch - 51

...

Epoch 29/30
468/468 - 44s - loss: 0.6307 - accuracy: 0.6405 - val_loss: 0.6177 - val_accuracy: 0.6587 - 44s/epoch - 95ms/step
Epoch 30/30
468/468 - 47s - loss: 0.6306 - accuracy: 0.6393 - val_loss: 0.6177 - val_accuracy: 0.6589 - 47s/epoch - 101ms/step
1170/1170 [==============================] - 8s 6ms/step
fold: 5 is complete
Subject:  sub-29
Epoch 1/30
468/468 - 49s - loss: 0.6853 - accuracy: 0.5467 - val_loss: 0.6246 - val_accuracy: 0.6661 - 49s/epoch - 104ms/step
Epoch 2/30
468/468 - 46s - loss: 0.5709 - accuracy: 0.6936 - val_loss: 0.5038 - val_accuracy: 0.7540 - 46s/epoch - 99ms/step
Epoch 3/30
468/468 - 44s - loss: 0.5265 - accuracy: 0.7344 - val_loss: 0.4792 - val_accuracy: 0.7671 - 44s/epoch - 93ms/step
Epoch 4/30
468/468 - 46s - loss: 0.5128 - accuracy: 0.7461 - val_loss: 0.4733 - val_accuracy: 0.7763 - 46s/epoch - 99ms/step
Epoch 5/30
468/468 - 46s - loss: 0.5047 - accuracy: 0.7509 - val_loss: 0.4611 - val_accuracy: 0.7810 - 46s/epoch - 99ms/step
Epoch 6/30
468/468 - 46s - 

Epoch 3/30
468/468 - 38s - loss: 0.5216 - accuracy: 0.7392 - val_loss: 0.4732 - val_accuracy: 0.7723 - 38s/epoch - 82ms/step
Epoch 4/30
468/468 - 42s - loss: 0.5090 - accuracy: 0.7487 - val_loss: 0.4662 - val_accuracy: 0.7774 - 42s/epoch - 91ms/step
Epoch 5/30
468/468 - 43s - loss: 0.4983 - accuracy: 0.7552 - val_loss: 0.4559 - val_accuracy: 0.7829 - 43s/epoch - 91ms/step
Epoch 6/30
468/468 - 43s - loss: 0.4935 - accuracy: 0.7593 - val_loss: 0.4508 - val_accuracy: 0.7880 - 43s/epoch - 91ms/step
Epoch 7/30
468/468 - 43s - loss: 0.4901 - accuracy: 0.7626 - val_loss: 0.4443 - val_accuracy: 0.7886 - 43s/epoch - 91ms/step
Epoch 8/30
468/468 - 43s - loss: 0.4866 - accuracy: 0.7648 - val_loss: 0.4430 - val_accuracy: 0.7911 - 43s/epoch - 91ms/step
Epoch 9/30
468/468 - 43s - loss: 0.4837 - accuracy: 0.7667 - val_loss: 0.4375 - val_accuracy: 0.7936 - 43s/epoch - 91ms/step
Epoch 10/30
468/468 - 42s - loss: 0.4818 - accuracy: 0.7682 - val_loss: 0.4389 - val_accuracy: 0.7939 - 42s/epoch - 91ms/step

Epoch 7/30
468/468 - 45s - loss: 0.4814 - accuracy: 0.7675 - val_loss: 0.4298 - val_accuracy: 0.8002 - 45s/epoch - 96ms/step
Epoch 8/30
468/468 - 45s - loss: 0.4790 - accuracy: 0.7688 - val_loss: 0.4234 - val_accuracy: 0.8036 - 45s/epoch - 96ms/step
Epoch 9/30
468/468 - 45s - loss: 0.4717 - accuracy: 0.7737 - val_loss: 0.4209 - val_accuracy: 0.8036 - 45s/epoch - 96ms/step
Epoch 10/30
468/468 - 45s - loss: 0.4696 - accuracy: 0.7758 - val_loss: 0.4198 - val_accuracy: 0.8089 - 45s/epoch - 96ms/step
Epoch 11/30
468/468 - 45s - loss: 0.4674 - accuracy: 0.7782 - val_loss: 0.4129 - val_accuracy: 0.8097 - 45s/epoch - 97ms/step
Epoch 12/30
468/468 - 45s - loss: 0.4650 - accuracy: 0.7786 - val_loss: 0.4074 - val_accuracy: 0.8115 - 45s/epoch - 97ms/step
Epoch 13/30
468/468 - 45s - loss: 0.4632 - accuracy: 0.7813 - val_loss: 0.4118 - val_accuracy: 0.8120 - 45s/epoch - 96ms/step
Epoch 14/30
468/468 - 35s - loss: 0.4603 - accuracy: 0.7818 - val_loss: 0.4068 - val_accuracy: 0.8112 - 35s/epoch - 76ms/

Epoch 11/30
468/468 - 44s - loss: 0.6688 - accuracy: 0.5900 - val_loss: 0.6662 - val_accuracy: 0.5976 - 44s/epoch - 93ms/step
Epoch 12/30
468/468 - 35s - loss: 0.6664 - accuracy: 0.5944 - val_loss: 0.6621 - val_accuracy: 0.6030 - 35s/epoch - 74ms/step
Epoch 13/30
468/468 - 44s - loss: 0.6659 - accuracy: 0.5949 - val_loss: 0.6616 - val_accuracy: 0.6029 - 44s/epoch - 95ms/step
Epoch 14/30
468/468 - 45s - loss: 0.6656 - accuracy: 0.5970 - val_loss: 0.6610 - val_accuracy: 0.6066 - 45s/epoch - 96ms/step
Epoch 15/30
468/468 - 45s - loss: 0.6644 - accuracy: 0.5975 - val_loss: 0.6585 - val_accuracy: 0.6075 - 45s/epoch - 96ms/step
Epoch 16/30
468/468 - 45s - loss: 0.6643 - accuracy: 0.5980 - val_loss: 0.6590 - val_accuracy: 0.6071 - 45s/epoch - 97ms/step
Epoch 17/30
468/468 - 46s - loss: 0.6629 - accuracy: 0.6012 - val_loss: 0.6576 - val_accuracy: 0.6077 - 46s/epoch - 97ms/step
Epoch 18/30
468/468 - 45s - loss: 0.6622 - accuracy: 0.6028 - val_loss: 0.6575 - val_accuracy: 0.6109 - 45s/epoch - 97

Epoch 15/30
468/468 - 47s - loss: 0.6662 - accuracy: 0.5965 - val_loss: 0.6599 - val_accuracy: 0.6090 - 47s/epoch - 100ms/step
Epoch 16/30
468/468 - 47s - loss: 0.6656 - accuracy: 0.5960 - val_loss: 0.6583 - val_accuracy: 0.6098 - 47s/epoch - 100ms/step
Epoch 17/30
468/468 - 47s - loss: 0.6653 - accuracy: 0.5972 - val_loss: 0.6574 - val_accuracy: 0.6097 - 47s/epoch - 99ms/step
Epoch 18/30
468/468 - 47s - loss: 0.6647 - accuracy: 0.5979 - val_loss: 0.6563 - val_accuracy: 0.6113 - 47s/epoch - 100ms/step
Epoch 19/30
468/468 - 47s - loss: 0.6646 - accuracy: 0.5978 - val_loss: 0.6573 - val_accuracy: 0.6115 - 47s/epoch - 100ms/step
Epoch 20/30
468/468 - 47s - loss: 0.6629 - accuracy: 0.5993 - val_loss: 0.6559 - val_accuracy: 0.6109 - 47s/epoch - 100ms/step
Epoch 21/30
468/468 - 46s - loss: 0.6629 - accuracy: 0.5992 - val_loss: 0.6548 - val_accuracy: 0.6131 - 46s/epoch - 98ms/step
Epoch 22/30
468/468 - 47s - loss: 0.6635 - accuracy: 0.6006 - val_loss: 0.6542 - val_accuracy: 0.6124 - 47s/epoch

## Save data

In [8]:
"""
Save training history and epoch level classification
"""
# Save accuracies history
with open(os.path.join(path, 'history2', 'EEG2Code_epoch_accuracies.txt'), 'w') as f:
    for acc in EEG2Code_acc_epoch:
        f.write('%s\n' % acc)
with open(os.path.join(path, 'history2', 'EEG2Code_trial_accuracies.txt'), 'w') as f:
    for acc in EEG2Code_acc_trial:
        f.write('%s\n' % acc)
        
# Save epoch predictions of all fold of all subjects
if not os.path.exists(os.path.join(path, 'history2')):
    os.makedirs(os.path.join(path, 'history2'))
np.save(os.path.join(path, 'history2', 'epoch_predictions_saveall_eeg2code.npy'), epoch_predictions_saveall_eeg2code)

### LDA, CCA+LDA

In [9]:
# Store accuracies for each subjects
LDA_acc_epoch = []
LDA_acc_trial = []
CCA_acc_epoch = []
CCA_acc_trial = []

# Store accuracies for each subjects

epoch_predictions_saveall_lda = np.empty((len(subjects),n_folds,int(n_trials/n_folds*(n_samples - epoch_size) / step_size)))
epoch_predictions_saveall_cca = np.empty((len(subjects),n_folds,int(n_trials/n_folds*(n_samples - epoch_size) / step_size)))

# Loop over all subjects
for i, subject in enumerate(subjects):
    X, V, y, fs = load_data(subject)
    print('Subject: ', subject)
    n_trials, n_channels, n_samples = X.shape
    # Set trial duration

    # Setup cross-validation
    folds = np.repeat(np.arange(n_folds), n_trials / n_folds)

    # Set up codebook for trial classification
    n = int(np.ceil(n_samples / V.shape[0]))
    _V = np.tile(V, (n, 1)).astype('float32')[:n_samples-epoch_size:step_size]

    # Setup LDAs
    lda = LinearDiscriminantAnalysis(solver='eigen', shrinkage='auto')
    lda2 = LinearDiscriminantAnalysis(solver='eigen', shrinkage='auto')
    
    # Setup CCA
    cca = CCA(n_components=1)
    
    # Loop folds
    accuracy_epoch_lda = np.zeros(n_folds)
    accuracy_trial_lda = np.zeros(n_folds)
    accuracy_epoch_cca = np.zeros(n_folds)
    accuracy_trial_cca = np.zeros(n_folds)
    
    # Store epoch predictions of all folds
    epoch_predictions_save_lda = np.empty((n_folds,int((n_samples - epoch_size) / step_size *n_trials/n_folds)))
    epoch_predictions_save_cca = np.empty((n_folds,int((n_samples - epoch_size) / step_size *n_trials/n_folds)))
    
    for i_fold in range(n_folds):

        # Split data to train and valid set
        X_trn, y_trn = X[folds != i_fold, :, :n_samples], y[folds != i_fold]
        X_tst, y_tst = X[folds == i_fold, :, :n_samples], y[folds == i_fold]
        
        # Slice trials to epochs
        X_sliced_trn, y_sliced_trn = trials_to_epochs(X_trn, y_trn, V, epoch_size, step_size)
        X_sliced_tst, y_sliced_tst = trials_to_epochs(X_tst, y_tst, V, epoch_size, step_size)
        
        """
        LDA method
        """
        
        # Train LDA (on epoch level)
        # N.B.: spatio-temporal features are flattened
        # N.B.: all epochs of all trials are concatenated
        lda.fit(X_sliced_trn.reshape((-1, n_channels * epoch_size)), y_sliced_trn.flatten())

        # Apply LDA (on epoch level)
        yh_sliced_tst_lda = lda.predict(X_sliced_tst.reshape((-1, n_channels * epoch_size)))
        
        # Append LDA epoch predictions for fold
        epoch_predictions_save_lda[i_fold] = np.array(yh_sliced_tst_lda.flatten())
        
        # Compute accuracy (on epoch level)
        accuracy_epoch_lda[i_fold] = 100 * np.mean(yh_sliced_tst_lda == y_sliced_tst.flatten())

        # Apply LDA (on trial level)
        ph_tst_lda = lda.predict_proba(X_sliced_tst.reshape((-1, n_channels * epoch_size)))[:, 1]
        ph_tst_lda = np.reshape(ph_tst_lda, y_sliced_tst.shape)
        rho_lda = pynt.utilities.correlation(ph_tst_lda, _V.T)
        yh_tst_lda = np.argmax(rho_lda, axis=1)
        accuracy_trial_lda[i_fold] = 100 * np.mean(yh_tst_lda == y_tst)
        
        """
        CCA+ LDA method
        """
        
        # Train CCA (on epoch level)
        erp_noflash = np.mean(X_sliced_trn[y_sliced_trn == 0, :, :], axis=0, keepdims=True)
        erp_flash = np.mean(X_sliced_trn[y_sliced_trn == 1, :, :], axis=0, keepdims=True)
        erps = np.concatenate((erp_noflash, erp_flash), axis=0)[y_sliced_trn, :, :]
        cca.fit(
            X_sliced_trn.transpose((0, 1, 3, 2)).reshape((-1, n_channels)), 
            erps.transpose((0, 1, 3, 2)).reshape(-1, n_channels))
        w = cca.x_weights_.flatten()
        
        # Apply CCA (on epoch level)
        X_sliced_filtered_trn = np.dot(w, 
            X_sliced_trn.transpose((2, 0, 1, 3)).reshape((n_channels, -1))).reshape(-1, epoch_size)
        X_sliced_filtered_tst = np.dot(w, 
            X_sliced_tst.transpose((2, 0, 1, 3)).reshape((n_channels, -1))).reshape(-1, epoch_size)
        
        # Train LDA (on epoch level)
        # N.B.: spatio-temporal features are flattened
        # N.B.: all epochs of all trials are concatenated
        lda2.fit(X_sliced_filtered_trn, y_sliced_trn.flatten())

        # Apply LDA (on epoch level)
        yh_sliced_tst_cca = lda2.predict(X_sliced_filtered_tst)
                                          
        # Append CCA+LDA epoch predictions for fold
        epoch_predictions_save_cca[i_fold] = np.array(yh_sliced_tst_cca.flatten())
                                          
        # Compute accuracy (on epoch level)
        accuracy_epoch_cca[i_fold] = 100 * np.mean(yh_sliced_tst_cca == y_sliced_tst.flatten())

        # Apply LDA (on trial level)
        ph_tst_cca = lda2.predict_proba(X_sliced_filtered_tst)[:, 1]
        ph_tst_cca = np.reshape(ph_tst_cca, y_sliced_tst.shape)
        rho_cca = pynt.utilities.correlation(ph_tst_cca, _V.T)
        yh_tst_cca = np.argmax(rho_cca, axis=1)
        accuracy_trial_cca[i_fold] = 100 * np.mean(yh_tst_cca == y_tst)
        
        print(f"fold: {i_fold+1} is complete")
        
    # Append the averaged fold accuracies for the subject
    LDA_acc_epoch.append(accuracy_epoch_lda.mean())
    LDA_acc_trial.append(accuracy_trial_lda.mean())
    CCA_acc_epoch.append(accuracy_epoch_cca.mean())
    CCA_acc_trial.append(accuracy_trial_cca.mean())
    
    # Append epoch predictions of all folds for subject
    epoch_predictions_saveall_lda[i] = epoch_predictions_save_lda
    epoch_predictions_saveall_cca[i] = epoch_predictions_save_cca

Subject:  sub-01
fold: 1 is complete
fold: 2 is complete
fold: 3 is complete
fold: 4 is complete
fold: 5 is complete
Subject:  sub-02
fold: 1 is complete
fold: 2 is complete
fold: 3 is complete
fold: 4 is complete
fold: 5 is complete
Subject:  sub-03
fold: 1 is complete
fold: 2 is complete
fold: 3 is complete
fold: 4 is complete
fold: 5 is complete
Subject:  sub-04
fold: 1 is complete
fold: 2 is complete
fold: 3 is complete
fold: 4 is complete
fold: 5 is complete
Subject:  sub-05
fold: 1 is complete
fold: 2 is complete
fold: 3 is complete
fold: 4 is complete
fold: 5 is complete
Subject:  sub-06
fold: 1 is complete
fold: 2 is complete
fold: 3 is complete
fold: 4 is complete
fold: 5 is complete
Subject:  sub-07
fold: 1 is complete
fold: 2 is complete
fold: 3 is complete
fold: 4 is complete
fold: 5 is complete
Subject:  sub-08
fold: 1 is complete
fold: 2 is complete
fold: 3 is complete
fold: 4 is complete
fold: 5 is complete
Subject:  sub-09
fold: 1 is complete
fold: 2 is complete
fold: 3

### Save data

In [10]:
"""
Save training history and epoch level classification
"""
# Save accuracies history
with open(os.path.join(path, 'history2', 'LDA_epoch_accuracies.txt'), 'w') as f:
    for acc in LDA_acc_epoch:
        f.write('%s\n' % acc)
with open(os.path.join(path, 'history2', 'LDA_trial_accuracies.txt'), 'w') as f:
    for acc in LDA_acc_trial:
        f.write('%s\n' % acc)
with open(os.path.join(path, 'history2', 'CCALDA_epoch_accuracies.txt'), 'w') as f:
    for acc in CCA_acc_epoch:
        f.write('%s\n' % acc)
with open(os.path.join(path, 'history2', 'CCALDA_trial_accuracies.txt'), 'w') as f:
    for acc in CCA_acc_trial:
        f.write('%s\n' % acc)
# Save epoch predictions of all fold of all subjects
np.save(os.path.join(path, 'history2', 'epoch_predictions_saveall_lda.npy'), epoch_predictions_saveall_lda)
np.save(os.path.join(path, 'history2', 'epoch_predictions_saveall_ldacca.npy'), epoch_predictions_saveall_cca)